<a href="https://colab.research.google.com/github/TheTrappist/teaching/blob/main/Biochem6761/AU24/02_curve_fit_explained_AU24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This notebook is designed to illustrate and describe the process of taking
# data and fitting them to a known equation, with step-by-step explanations of
# the functions used. Please read these comments throughout the code; they will
# provide guidance along the way!

In [ ]:
# First, let's download the data used for this exercise

import pandas as pd # The pandas library will just be used for reading csv files

# The following command will parse the file and store it in the variable "data"
data = pd.read_csv('https://raw.githubusercontent.com/TheTrappist/teaching/main/Biochem6761/AU24/Sample_data/AU24_05_bacterial_growth_data.csv')


In [ ]:
# The pandas read_csv command simply takes the data file and saves it to a
# special type of variable called a "dataframe". Think of a dataframe as a
# table, like a table you'd find in Excel or published in a paper. Let's print
# it to see what the whole dataset looks like:

print(data)

What you see in the table above looks just like a table, right? These data represent the growth of two different bacterial cultures in early log phase (where it can be well-modeled as exponential growth). The first column, time_min, contains the time (in minutes) at which the density (OD) of cultures was measured, while the columns od_culture1 and od_culture2 store the OD measurements of the two cultures at each time point.

In [ ]:
# How do we select just one column of a dataframe? It's very easy: you just need
# the column name. Let's try:

# Select the first column
col1 = data['time_min']
# Note: the square brackets are needed to show that we're selecting a subset
# (a fragment) of 'data'.

# Let's make sure it worked by printing the result
print(col1)

In [ ]:
# Now, let's select the column storing optical densities of culture 2:
culture2 = data['od_culture2']

print(culture2)

In [ ]:
# Now, let's make a simple plot of culture2 optical densities at each time point
from matplotlib import pyplot as plt

plt.style.use("bmh") # This is emphatically NOT required, it's just a visual
# option to make plots look a little nicer. Try removing it to see what happens.

# First, we need to create a figure and an axes object to plot on:

fig, ax = plt.subplots() # the variable "ax" that we got here is the name of the
# new axes object that we can make one or more plots in.

# Let's plot OD vs. time for culture 1
time = data['time_min']
od_cult1 = data['od_culture1']

ax.plot(time, od_cult1, ls='', marker='o') # This is the main plot command.
# It requires x-values ('time' in our case), y-values ('od_cult1'), and can also
# accept a range of different optional parameters. I am passing two optional
# parameters to it here: 'ls', which stands for linestyle and specifies how the
# individual points of the plot are to be connected, and 'marker', which is the
# way individual points are to be represented. I am setting ls to an empty
# string ('') to show that I don't want points to be connected by a line at all,
# and I'm setting marker to 'o' to make it into circles. Try plugging different
# values for these and re-running the code to see what happens (e.g. '-' for ls
# and '.' or 'x' for marker)

In [ ]:
# Great, we have a plot! Now comes the fitting part. To fit, we need to
# define an equation we'll be using to fit. We're assuming exponential growth
# here, so let's fit it to an exponential equation of the form y = a*2^(x/tau).
# This way, the parameter a will be a scaling factor (pre-exponent) while tau
# will be a characteristic time parameter (effectively, the doubling time of the
# culture).

# To define a custom equation, we need to use the following syntax:

def exp_growth(x, a, tau): # def is a key word indicating a new function
  # The things in parenthesis in the function definition are all the different
  # values that you need to evaluate the function. In our case, that's three
  # values: x, which is the independent variable (time), the pre-exponent a,
  # and the characteristic time tau.

  #The actual function goes here
  y = a*2**(x/tau) # double star (**) means "raise to a power"
  return y # This is what the function gives back to us after it's evaluated.

# We have the function defined now! But we haven't asked it to do anything yet.

In [ ]:
# Now, let's fit our data to the exponential function we defined

from scipy.optimize import curve_fit # We need to import the curve_fit function

# to use curve_fit, we need to give it the function to fit to, x-values,
# y-values, and some optional parameters.
popt, pcov = curve_fit(exp_growth, time, od_cult1) # This part does the fitting!

# The fit already happened, but we don't know how it performed yet. The results
# of the fit are now stored int he variable 'popt'. Let's print this variable
# and investigate:

print(popt)

# As you can see, there are two values in popt. That's because we are fitting a
# function that takes two parameters in addition to x values; specifically,
# these parameters are 'a' and 'tau'. The 'popt' variable is an array that has
# as many values as the number of free parameters in the function you're using.
# How do we know which one is which? It's simple: they appear in popt in the
# same order as they did in our function definition.

In [ ]:
# This means that the first element in popt should have the fit result for the
# pre-exponent, and the second element should have the fit result for the
# characteristic time tau. Let's see what the pre-exponent looks like:

print(popt[0])

In [ ]:
# That's a REALLY small value, so our fitting probably failed. However, to be
# sure, we can just plot the fit result. To do that, we need to generate a new
# curve for the fit result.

# Generating a new curve starts with making a number of closely spaced x-values.

import numpy as np # Numpy is a library we will need for handling arrays of data

x_vals = np.linspace(0,120,100) # The linspace command makes any number of
# equally spaced values between two endpoints. The first two parameters in
# parentheses indicate the start and end points, while the third parameter is
# the number of datapoints we want.

# Now that we have 100 x-values from 0 to 120 minutes, let's calculate the
# y-values for them using our (probably wrong) fit results.

a = popt[0] # pre-exponent from our fit
tau = popt [1] # tau from our fit

y_vals_fit = exp_growth(x_vals, a, tau) # Just using our previous function!

# Now, we have paired x and y values. Let's plot them next to the data to see
# how it all looks

fig2, ax2 = plt.subplots() # Create the plot
ax2.plot(time, od_cult1, ls='', marker='o') # Plot the raw data
ax2.plot(x_vals, y_vals_fit) # plot the fit result

In [ ]:
# As you can see, the plotting worked, but our fit is clearly not good since it
# explodes to insanely high values at the 120 minute mark. This is sometimes a
# problem with nonlinear curve fitting, where a bad initial guess of the fit
# parameters can get the algorithm stuck in an unphysical local minimum. We can
# help the algorithm do much better by giving it coarse but realistic estimates
# of tau and the pre-exponent, which we can get by looking at our plot. To do
# this, we pass the optional 'p0' parameter to the curve_fit function; p0 should
# be a list with the same number of elements as the number of free parameters in
# the fit function (in our case, two: one for pre-exponent, one for tau)

popt, pcov = curve_fit(exp_growth, time, od_cult1, p0=[0.1, 50])

print(popt)

In [ ]:
# Now the two values are looking much more realistic! Let's plot the new fit.

y_vals_newfit = exp_growth(x_vals, popt[0], popt[1]) # Getting new popt values


fig3, ax3 = plt.subplots() # Create the plot
ax3.plot(time, od_cult1, ls='', marker='o') # Plot the raw data
ax3.plot(x_vals, y_vals_newfit) # plot the fit result

In [ ]:
# Looks so much better, doesn't it? Now let's put it all together and fit the
# two datasets side by side

od_cult2 = data['od_culture2']

popt2, pcov2 = curve_fit(exp_growth, time, od_cult2, p0=[0.1, 50]) # new fit
y_vals_cult2 = exp_growth(x_vals, popt2[0], popt2[1]) # Getting new popt values

fig4, ax4 = plt.subplots() # Create the plot
ax4.plot(time, od_cult1, ls='', marker='o') # Plot the raw data
ax4.plot(x_vals, y_vals_newfit) # plot the fit result

ax4.plot(time, od_cult2, ls='', marker='o') # Plot the raw data for culture 2
ax4.plot(x_vals, y_vals_cult2) # plot the fit result for culture 2

In [ ]:
# Finally, let's print the characteristic times from culture 1 and culture 2:

print(popt[1]) # Characteristic time from culture 1
print(popt2[1]) # Characteristic time from culutre 2

# The time (in minutes) from culture 2 is almost twice as slow as that from
# culture 1. Does that match what you see on the plot?